In [ ]:
%load_ext autoreload
%autoreload 2

from tcd_pipeline.modelrunner import ModelRunner
import ttach as tta
from PIL import Image
import numpy as np
import os
import glob
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#!pip install python-liquid

def render_report(output_path, report_results):
    from liquid import Environment
    from liquid import Mode
    from liquid import StrictUndefined
    from liquid import FileSystemLoader

    env = Environment(
        tolerance=Mode.STRICT,
        undefined=StrictUndefined,
        loader=FileSystemLoader("../templates/"),
    )

    template = env.get_template("canopy_coverage.html")
    result = template.render(data=report_results, page_title="Canopy coverage report")

    with open(os.path.join(output_path, "report.html"), 'w') as fp:
        fp.write(result)

def predict_and_report(runner, image_path):
    
    results = runner.predict(image_path, warm_start = True)
    file_stem = Path(image_path).stem
    report_folder = f"./results_{file_stem}"
    
    os.makedirs(report_folder, exist_ok=True)
    
    results.save_masks(output_path=report_folder, image_path=image_path)
    results.visualise(output_path=report_folder)
    
    report_results = {
        "raw_image": "raw_image.jpg",
        "canopy_mask": "canopy_mask.jpg",
        "canopy_confidence": "confidence_map.jpg",
        "local_maximum": "local_maximum.jpg",
        "canopy_cover": results.canopy_cover,
        "image_name": os.path.basename(results.image.name),
        "image_res": results.image.res[0],
        "image_crs": results.image.crs,
        "config": runner.config
    }
    
    render_report(report_folder, report_results)

In [ ]:
runner = ModelRunner("../config/base_semantic_segmentation.yaml")
runner.model.model = tta.SegmentationTTAWrapper(runner.model.model, transforms=tta.aliases.d4_transform())

In [ ]:
image_paths = glob.glob("../data/")

In [ ]:
for image_path in image_paths:
    predict_and_report(runner, image_path)